# Generate data with small beads and Poisson noise from experimental PSF

In [1]:
import numpy as np
import tifffile

In [2]:
import napari

In [4]:
#Create a 2d array to do FFT
ashape = (256,256,256)

a = np.zeros(ashape, dtype=float)

#Add a few cubes in grid-like locations
cubesize=2
cubespacing=60
for iz in range(int(cubespacing/2),ashape[0],cubespacing):
    for iy in range(int(cubespacing/2),ashape[1],cubespacing):
        for ix in range(int(cubespacing/2),ashape[2],cubespacing):
            a[iz:iz+cubesize , iy:iy+cubesize , ix:ix+cubesize] = np.ones((cubesize,cubesize,cubesize))

In [5]:
nview_data  = napari.view_image(a, ndisplay=3)
#OK

In [ ]:
#Optionally save the data
tifffile.imsave('test/gendata1_raw.tif', a)

## Convolve data with the experimental 'Rosalind' Psf.

Read data first

In [8]:
psfdata=tifffile.imread('PSF_RFI_8bit.tif')

In [9]:
type(psfdata)

numpy.ndarray

In [10]:
psfdata.dtype

dtype('uint8')

In [11]:
psfdata.shape

(61, 32, 32)

In [12]:
psfdata_norm = (psfdata.astype(float) - psfdata.min() ) / (psfdata.max() - psfdata.min())

In [13]:
nview_psf = napari.view_image(psfdata_norm, ndisplay=3)

Exception in callback BaseAsyncIOLoop._handle_events(4036, 1)
handle: <Handle BaseAsyncIOLoop._handle_events(4036, 1)>
Traceback (most recent call last):
  File "C:\Users\Luis\miniconda3\envs\dev\lib\asyncio\events.py", line 81, in _run
    self._context.run(self._callback, *self._args)
  File "C:\Users\Luis\miniconda3\envs\dev\lib\site-packages\tornado\platform\asyncio.py", line 189, in _handle_events
    handler_func(fileobj, events)
  File "C:\Users\Luis\miniconda3\envs\dev\lib\site-packages\zmq\eventloop\zmqstream.py", line 452, in _handle_events
    self._handle_recv()
  File "C:\Users\Luis\miniconda3\envs\dev\lib\site-packages\zmq\eventloop\zmqstream.py", line 481, in _handle_recv
    self._run_callback(callback, msg)
  File "C:\Users\Luis\miniconda3\envs\dev\lib\site-packages\zmq\eventloop\zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "C:\Users\Luis\miniconda3\envs\dev\lib\site-packages\jupyter_client\threaded.py", line 121, in _handle_recv
    m

## Convolve

In [14]:
import scipy.signal

In [15]:
data_convolved = scipy.signal.convolve(a, psfdata_norm, mode='same')

In [16]:
data_convolved.shape

(256, 256, 256)

In [28]:
#normalises to 0-255 range
data_convolved = (data_convolved - data_convolved.min()) / (data_convolved.max() - data_convolved.min())*255

In [29]:
print(data_convolved.max())
print(data_convolved.min())

255.0
0.0


In [36]:
nview_dataconv = napari.view_image(data_convolved,ndisplay=3)

## Add Poisson noise

In [30]:
#data_convolved_noised = data_convolved + np.random.poisson(256 , size=ashape).astype(np.float32)/80
#This method of adding does not look right. The original intensity should be the lambda poisson parameter in the function

In [31]:
rng = np.random.default_rng()

In [34]:
data_convolved_noised = rng.poisson(lam = data_convolved)

In [35]:
nview_data_noised = napari.view_image(data_convolved_noised,ndisplay=3)

In [41]:
data_convolved_noised_uint8 = ((data_convolved_noised - data_convolved_noised.min()) / ( data_convolved_noised.max() - data_convolved_noised.min() ) *255 ).astype(np.uint8)

In [43]:
tifffile.imsave('test/gendata_psfconv_poiss.tif', data_convolved_noised_uint8)

# Create large data

In [1]:
import numpy as np
import tifffile
import napari
import scipy.signal

#Create a 2d array to do FFT
ashape = (60,1026,1544) # Casper LM size
a = np.zeros(ashape, dtype=float)
#a = np.random.poisson(256 , size=(size0,size0,size0)).astype(np.float32)/2000

#Add a few cubes in grid-like locations
cubesize=2
cubespacing=67
for iz in range(5,ashape[0],cubespacing):
    for iy in range(5,ashape[1],cubespacing):
        for ix in range(5,ashape[2],cubespacing):
            a[iz:iz+cubesize , iy:iy+cubesize , ix:ix+cubesize] = np.ones((cubesize,cubesize,cubesize))

#Read psf
psfdata=tifffile.imread('PSF_RFI_8bit.tif')
psfdata_norm = (psfdata.astype(float) - psfdata.min() ) / (psfdata.max() - psfdata.min())

#Convolve
data_convolved = scipy.signal.convolve(a, psfdata_norm, mode='same')

#Adjust max/min and intensity
data_convolved = (data_convolved - data_convolved.min()) / (data_convolved.max() - data_convolved.min())*255

#Noisify with Poisson
rng = np.random.default_rng()
data_convolved_noised = rng.poisson(lam = data_convolved)
data_convolved_noised_uint8 = ((data_convolved_noised - data_convolved_noised.min()) / ( data_convolved_noised.max() - data_convolved_noised.min() ) *255 ).astype(np.uint8)

In [ ]:
tifffile.imsave('gendata_psfconv_poiss_large.tif', data_convolved_noised_uint8)

# Create very large data

In [4]:
import numpy as np
import tifffile
import napari
import scipy.signal

#Create a 2d array to do FFT
ashape = (51,2048,2048) # Jeonyoon Choi
a = np.zeros(ashape, dtype=float)
#a = np.random.poisson(256 , size=(size0,size0,size0)).astype(np.float32)/2000

#Add a few cubes in grid-like locations
cubesize=2
cubespacing=67
for iz in range(5,ashape[0],cubespacing):
    for iy in range(5,ashape[1],cubespacing):
        for ix in range(5,ashape[2],cubespacing):
            a[iz:iz+cubesize , iy:iy+cubesize , ix:ix+cubesize] = np.ones((cubesize,cubesize,cubesize))

#Read psf
psfdata=tifffile.imread('PSF_RFI_8bit.tif')
psfdata_norm = (psfdata.astype(float) - psfdata.min() ) / (psfdata.max() - psfdata.min())

#Convolve
data_convolved = scipy.signal.convolve(a, psfdata_norm, mode='same')

#Adjust max/min and intensity
data_convolved = (data_convolved - data_convolved.min()) / (data_convolved.max() - data_convolved.min())*255

#Noisify with Poisson
rng = np.random.default_rng()
data_convolved_noised = rng.poisson(lam = data_convolved)
data_convolved_noised_uint8 = ((data_convolved_noised - data_convolved_noised.min()) / ( data_convolved_noised.max() - data_convolved_noised.min() ) *255 ).astype(np.uint8)

In [ ]:
nview_data_noised = napari.view_image(data_convolved_noised_uint8,ndisplay=3)

In [3]:
tifffile.imsave('gendata_psfconv_poiss_vlarge.tif', data_convolved_noised_uint8)

NameError: name 'data_convolved_noised_uint8' is not defined